<a href="https://colab.research.google.com/github/VIOLDAVE/Massive-Algorithm/blob/main/Market.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install polars pyfpgrowth mlxtend spacy datasketch pyvis #rapids-cuml

  Using cached datasketch-1.6.5-py3-none-any.whl.metadata (5.8 kB)
  Using cached pyvis-0.3.2-py3-none-any.whl.metadata (1.7 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.2/89.2 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 53.0 MB/s eta 0:00:00


In [ ]:
!pip install pyfpgrowth==1.0

In [ ]:

import os
import polars as pl
from pathlib import Path

In [ ]:
# Kaggle Authentication
os.environ["KAGGLE_USERNAME"] = "aworviola2"
os.environ["KAGGLE_KEY"] = "b10218d275dad1e914008b49da4f407d"
!mkdir -p ~/.kaggle
!echo '{"username":"aworviola2","key":"b10218d275dad1e914008b49da4f407d"}' > ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json  # Critical permission setting


In [ ]:
dataset = "arashnic/book-recommendation-dataset"
data_dir = Path("data")
data_dir.mkdir(exist_ok=True)

Data downloading

In [ ]:

!kaggle datasets download -d mohamedbakhet/amazon-books-reviews -p {data_dir} --unzip
df = pl.scan_csv(data_dir / "Books_rating.csv").collect()

Dataset URL: https://www.kaggle.com/datasets/mohamedbakhet/amazon-books-reviews
License(s): CC0-1.0
 99% 1.05G/1.06G [00:03<00:00, 283MB/s]
100% 1.06G/1.06G [00:04<00:00, 279MB/s]


Explorartory data analysis

In [ ]:
display(df.head())

Id,Title,Price,User_id,profileName,review/helpfulness,review/score,review/time,review/summary,review/text
str,str,f64,str,str,str,f64,i64,str,str
"""1882931173""","""Its Only Art If Its Well Hung!""",null,"""AVCGYZL8FQQTD""","""Jim of Oz ""jim-of-oz""""","""7/7""",4.0,940636800,"""Nice collection of Julie Strai…","""This is only for Julie Strain …"
"""0826414346""","""Dr. Seuss: American Icon""",null,"""A30TK6U7DNS82R""","""Kevin Killian""","""10/10""",5.0,1095724800,"""Really Enjoyed It""","""I don't care much for Dr. Seus…"
"""0826414346""","""Dr. Seuss: American Icon""",null,"""A3UH4UZ4RSVO82""","""John Granger""","""10/11""",5.0,1078790400,"""Essential for every personal a…","""If people become the books the…"
"""0826414346""","""Dr. Seuss: American Icon""",null,"""A2MVUWT453QH61""","""Roy E. Perry ""amateur philosop…","""7/7""",4.0,1090713600,"""Phlip Nel gives silly Seuss a …","""Theodore Seuss Geisel (1904-19…"
"""0826414346""","""Dr. Seuss: American Icon""",null,"""A22X4XUPKF66MR""","""D. H. Richards ""ninthwavestore…","""3/3""",4.0,1107993600,"""Good academic overview""","""Philip Nel - Dr. Seuss: Americ…"


In [ ]:
display(df.describe())

statistic,Id,Title,Price,User_id,profileName,review/helpfulness,review/score,review/time,review/summary,review/text
str,str,str,f64,str,str,str,f64,f64,str,str
"""count""","""3000000""","""2999792""",481171.0,"""2438213""","""2438114""","""3000000""",3e6,3e6,"""2999962""","""2999992"""
"""null_count""","""0""","""208""",2.518829e6,"""561787""","""561886""","""0""",0.0,0.0,"""38""","""8"""
"""mean""",null,null,21.762656,null,null,null,4.215289,1.1323e9,null,null
"""std""",null,null,26.206541,null,null,null,1.203054,1.4932e8,null,null
"""min""","""0001047604""",""""" Film technique, "" and, "" Fil…",1.0,"""A00109803PZJ91RLT7DPN""","""""","""0/0""",1.0,-1.0,"""!""","""The Tao of Muhammad Ali is t…"
"""25%""",null,null,10.78,null,null,null,4.0,9.999072e8,null,null
"""50%""",null,null,14.93,null,null,null,5.0,1.1283e9,null,null
"""75%""",null,null,23.95,null,null,null,5.0,1.2691e9,null,null
"""max""","""B0064P287I""","""you can do anything with crepe…",995.0,"""AZZZZW74AAX75""","""~~~~~~~""","""997/1090""",5.0,1.3624e9,"""~~~~~~~~~~~~~~~~~~~~~~~~~~""","""~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~…"


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df.null_count()

Id,Title,Price,User_id,profileName,review/helpfulness,review/score,review/time,review/summary,review/text
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,208,2518829,561787,561886,0,0,0,38,8


In [ ]:
display(df.is_duplicated().sum())

16483

AttributeError: 'DataFrame' object has no attribute 'is_null'